Materialize query history system table for best performance, and so that primary key optimizations can be applied.

See [query optimization using primary key constraints](https://docs.databricks.com/aws/en/sql/user/queries/query-optimization-constraints) for more details.

In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
create or replace table fct_usage(
  calendar_key int comment 'FK for calendar dimension (dim_calendar).',
  compute_key string comment 'FK for compute dimension (dim_compute).',
  workspace_key string comment 'FK for workspace dimension (dim_workspace).',
  record_id string comment 'Unique ID for this usage record',
  sku_name string comment 'Name of the SKU',
  cloud string comment 'Cloud this usage is relevant for. Possible values are AWS, AZURE, and GCP.',
  usage_start_time timestamp comment 'The start time relevant to this usage record. Timezone information is recorded at the end of the value with +00:00 representing UTC timezone.',
  usage_end_time timestamp comment 'The end time relevant to this usage record. Timezone information is recorded at the end of the value with +00:00 representing UTC timezone.',
  usage_date date comment 'Date of the usage record, this field can be used for faster aggregation by date',
  usage_unit string comment 'Unit this usage is measured in. Possible values include DBUs.',
  usage_quantity decimal(38, 18) comment 'Number of units consumed for this record.',
  record_type string comment 'Whether the record is original, a retraction, or a restatement. The value is ORIGINAL unless the record is related to a correction. See [Analyze correction records](https://docs.databricks.com/en/admin/system-tables/billing.html#record-type).',
  ingestion_date date comment 'Date the record was ingested into the usage table.',
  billing_origin_product string comment 'The product that originated the usage. Some products can be billed as different SKUs. For possible values, see [View information about the product associated with the usage](https://docs.databricks.com/en/admin/system-tables/billing.html#features).',
  usage_type string comment 'The type of usage attributed to the product or workload for billing purposes. Possible values are COMPUTE_TIME, STORAGE_SPACE, NETWORK_BYTES, API_CALLS, TOKEN, or GPU_TIME.',
  constraint fk_fct_usage_dim_calendar
    foreign key (calendar_key) references dim_calendar (calendar_key),
  constraint fk_fct_usage_dim_compute
    foreign key (compute_key) references dim_compute (compute_key),
  constraint fk_fct_usage_dim_workspace
    foreign key (workspace_key) references dim_workspace (workspace_key)
)
cluster by (calendar_key, compute_key, workspace_key);

In [0]:
-- Returns usage associated with DBSQL warehouses
insert overwrite fct_usage
select
  year(usage_date) * 10000 + month(usage_date) * 100 + day(usage_date) as calendar_key,
  usage_metadata.warehouse_id as compute_key,
  workspace_id as workspace_key,
  record_id,
  sku_name,
  cloud,
  usage_start_time,
  usage_end_time,
  usage_date,
  usage_unit,
  usage_quantity,
  record_type,
  ingestion_date,
  billing_origin_product,
  usage_type
from system.billing.usage
where usage_date >= current_date() - interval 6 months
and usage_metadata.warehouse_id is not null;

In [0]:
optimize fct_usage

In [0]:
analyze table fct_usage compute statistics for all columns;

In [0]:
vacuum fct_usage